In [276]:
import pandas as pd

# Importing crime data from each year and creating one large data file

#### Looking at what is in each file using 2012 as a baseline in order to decide what to merge and how

In [277]:
Incident_2012 = pd.read_csv('../theft_and_crime_clt/Data_input/Incident/2012_Incident.csv')
Incident_2012.head()

,Complaint_No,Block_No,Direction,Street_Name,Street_Type,Suffix,Apt_No,City,State,Zipcode,...,Incident_to_date,Incident_to_Time,Place1,Place2,Clearance_Status,Clearance_Date,Case_Status,Reporting_Agency,Follow_up_Section,NIBRS_Hi_Class
0,20120616144301,9108,,REAMES,RD,,,CHARLOTTE,NC,28216,...,6/16/12,1330.0,Residential,Residential Care Facility ...,Exceptionally Cleared - Located (Missing Perso...,6/16/12,Close/Cleared,Charlotte-Mecklenburg Police Department,70-FIB - MISSING PERSONS ...,Missing Person ...
1,20120616144400,2903,N,DAVIDSON,ST,,,CHARLOTTE,NC,28205,...,6/16/12,1230.0,Residential,Apartment/Duplex Private Res ...,Open ...,,Further Investigation,Charlotte-Mecklenburg Police Department,06-EASTWAY DIVISION ...,Theft of Motor Vehicle Parts from Vehicle ...
2,20120616144700,601,W,24TH,ST,,,CHARLOTTE,NC,28206,...,NaN,NaN,Commercial Place,Other - Commercial Place ...,Open ...,,Inactive,Charlotte-Mecklenburg Police Department,02-METRO DIVISION ...,All Other Thefts ...
3,20120616145104,5522,,LABORDE,AV,,,CHARLOTTE,NC,28269,...,6/15/12,1635.0,Residential,Private Residence ...,Open ...,,Inactive,Charlotte-Mecklenburg Police Department,11-NORTH DIVISION ...,Damage/Vandalism Of Property ...
4,20120616145201,7718,,PRINDLE LAKE,DR,,,CHARLOTTE,NC,28227,...,6/16/12,1441.0,Residential,Private Residence ...,Exceptionally Cleared - Victim Chose not to Pr...,6/18/12,Close/Cleared,Charlotte-Mecklenburg Police Department,12-HICKORY GROVE DIVISION ...,Intimidation ...


In [278]:
Offenses_2012 = pd.read_csv('../theft_and_crime_clt/Data_input/Offense/2012_Offenses.csv')
Offenses_2012.head()

,Complaint_No,off_Attempt_Complete,Off_nibrs_crime_category,off_nibrs_classification,off_noc_code
0,20120616144301,Completed,Missing Person / Missing Person ...,Missing Person ...,800
1,20120616144400,Completed,Larceny / Larceny: Misdemeanor ...,Theft of Motor Vehicle Parts from Vehicle ...,23G
2,20120616144700,Completed,Damage to property / Injury to Real or Persona...,All Other Thefts ...,290
3,20120616144700,Completed,Larceny / Larceny: Misdemeanor ...,All Other Thefts ...,23H
4,20120616145104,Completed,Damage to property / Injury to Real or Persona...,Damage/Vandalism Of Property ...,290


In [279]:
print(Offenses_2012['Off_nibrs_crime_category'].nunique())
print(Offenses_2012['off_nibrs_classification'].nunique())
print(Incident_2012['NIBRS_Hi_Class'].nunique())

119
64
64


Offenses has an additional level of crime classification category with Off_nibrs_crime_category, while Incidents only has nibrs_hi_class. Will keep both because we plan to self-define property crimes.

In [280]:
Property_2012 = pd.read_csv('../theft_and_crime_clt/Data_input/Property/2012_Property.csv')
Property_2012.head()

,Complaint_No,Property_No,Property_type,Property_Desc
0,20120616144400,1,Vehicle Tag ...,Vehicles and Parts ...
1,20120616144700,1,Other ...,Miscellaneous ...
2,20120616144700,2,Other ...,Miscellaneous ...
3,20120616145104,1,Dwelling: Single-Family ...,Structures ...
4,20120616145203,1,Purses/Handbags/Wallets ...,Money and Securities ...


In [281]:
Property_2012['Property_No'].describe()

count    203577.000000
mean          3.440271
std           6.246846
min           1.000000
25%           1.000000
50%           2.000000
75%           4.000000
max         192.000000
Name: Property_No, dtype: float64

Some crimes have a lot of impacted property items, with the average being 3.44, and the maximum being 192. Since we are focused There may be multiple property instances per crime if more than one thing was damaged or taken -- how do we want to handle this?

In [282]:
Property_2012['Property_Desc'].unique()

array(['Vehicles and Parts                                ',
       'Miscellaneous                                     ',
       'Structures                                        ',
       'Money and Securities                              ',
       'Other Property                                    ',
       'Withheld                                          ',
       'Electronic Equipment and Accessories              ',
       'Jewelry and Collectibles                          ',
       'Consumable Goods                                  ',
       'Building Materials                                ',
       'Electronic Equipment and Accesssories             ',
       'Crops and Landscaping                             ',
       'Animals                                           '], dtype=object)

In [283]:
Property_2012['Property_type'].unique()

array(['Vehicle Tag                                       ',
       'Other                                             ',
       'Dwelling: Single-Family                           ',
       'Purses/Handbags/Wallets                           ',
       'Credit/Debit Cards                                ',
       'Money/Cash                                        ',
       'Windows                                           ',
       'Withheld                                          ',
       'Telephone Equipment                               ',
       'Tools (Power and Hand)                            ',
       'TV Equipment                                      ',
       'Video Game Systems and Accessories                ',
       'Computer Hardware/Software                        ',
       'Camera/Optical Equipment                          ',
       'Doors/Garage Doors                                ',
       'Jewelry                                           ',
       'Vehicle Parts/Ac

Only look at the property_desc high level list. 

Property_desc is a higher level category of property involved, while Property_type gets much more specific

In [284]:
Vehicle_2012 = pd.read_csv('../theft_and_crime_clt/Data_input/Vehicle/2012_Stolen_Vehicle.csv')
Vehicle_2012.head()

,Complaint_No,Vehicle_Year,Vechicle_Make,Vehicle_Model,Vehicle_Color,Vehicle_Body
0,20120616212404,2004,Buick ...,CENTURY,Silver/Silver ...,4-Door ...
1,20120617102701,1997,Chrysler ...,TOWN AND COUNTRY,White/White ...,Van ...
2,20120617153800,1999,Chevrolet ...,CAMARO,Green/Green ...,2-Door ...
3,20120617213000,1999,Oldsmobile ...,CUTLASS,White/White ...,4-Door ...
4,20120617215500,2006,Hyundai ...,SONATA,Tan/Tan ...,4-Door ...


Vehicles are only for stolen vehicles -- do we want to include these and consider them a type of property? Currently not included in property file (above) because they are considered a unique class.

In [285]:
Business_2012 = pd.read_csv('../theft_and_crime_clt/Data_input/Business/2012_Victim_Business.csv')
Business_2012.head()

,Complaint_No,Victim_Bus_No,Victim_Bus_LegalName,Victim_Bus_Name,Victim_Bus_Type,Victim_Bus_Block_No,Victim_Bus_Direction,Victim_Bus_Street_Name,Victim_Bus_Suffix,Victim_Bus_Apt_No1,Victim_Bus_Apt_No2,Victim_Bus_City,Victim_Bus_State,Victim_Bus_ZipCode
0,20120616144700,1,COMPRESSOR SERVICE CO. ...,COMPRESSOR SERVICE CO. ...,Business ...,601,W,24TH,ST,,,CHARLOTTE,NC,28206
1,20120616151205,1,THE UNIVERSAL CHURCH ...,THE UNIVERSAL CHURCH ...,Religious Organization ...,5000,N,TRYON,ST,,,CHARLOTTE,NC,
2,20120616155402,1,STATE OF NORTH CAROLINA ...,STATE OF NORTH CAROLINA ...,Society/Public ...,,,,,,,,,
3,20120616160600,1,CITGO TIMES INC ...,CITGO TIMES INC ...,Business ...,8400,,OLD STATESVILLE,RD,,,CHARLOTTE,NC,28269
4,,2,STATE OF NORTH CAROLINA ...,STATE OF NORTH CAROLINA ...,Society/Public ...,,,,,,,,,


In [286]:
Business_2012['Victim_Bus_Type'].nunique()

129

In [287]:
Incident_2012['Place2'].nunique()

68

In [288]:
Incident_2012['Place1'].nunique()

5

Business includes the name of the business impacted (or State of NC for public crimes) as well as a more specific classification of business type with 129 unique values for Victim_Bus_Type rather than the 68 in Place2 of Incident.

In [289]:
Person_2012 = pd.read_csv('../theft_and_crime_clt/Data_input/Person/2012_Victim_Person.csv')
Person_2012.head()

,Complaint_No,Victim_No,Victim_Age,Victim_Race,Victim_Gender,Victim_Ethnicity
0,20120616144301,1,50,Black,M,Other/Not Listed ...
1,20120616144400,1,26,White,F,Unknown/Refused ...
2,20120616145104,1,35,Black,F,Other/Not Listed ...
3,20120616145201,1,26,Black,M,Unknown/Refused ...
4,20120616145203,1,62,White,F,Other/Not Listed ...


Victim is personal information about the victim. Will not be needed.

In [290]:
Weapon_2012 = pd.read_csv('../theft_and_crime_clt/Data_input/Weapons/2012_Weapons.csv')
Weapon_2012.head()

,Complaint_No,Weapon_Type
0,20120616155200,"Personal Weapons (hands,feet,teeth,etc.)"
1,20120616163800,"Personal Weapons (hands,feet,teeth,etc.)"
2,20120616163800,Other Weapon
3,20120616165800,"Personal Weapons (hands,feet,teeth,etc.)"
4,20120616170401,"Personal Weapons (hands,feet,teeth,etc.)"


Weapon file includes information about weapons will not be needed.

## Final merging plan:

 Keep all information from incident to start, then add in Off_nibrs_crime_category from Offenses. Because this is about property, we will merge left with all property information on crime_no, leaving us with a row for each item impacted by a crime.

If we want to include vehicles, we can also concat all of the vehicle thefts and their relavant incidents, creating a new Property_Desc and Property_type of Vehicle (noting here that automobile is already a category in Property_type should be investigated)

For Victime_Bus, pull in business names as well as the more specific Victime_Bus_Type

Do not pull in anything from victim or weapon.

### Reading in all the files into 4 larger data frames

Combine years into each data frame

In [291]:
#make a list of files in the directory
import os
incident_list = os.listdir('../theft_and_crime_clt/Data_input/Incident/')
offense_list = os.listdir('../theft_and_crime_clt/Data_input/Offense/')
property_list = os.listdir('../theft_and_crime_clt/Data_input/Property/')
vehicle_list = os.listdir('../theft_and_crime_clt/Data_input/Vehicle/')
business_list = os.listdir('../theft_and_crime_clt/Data_input/Business/')


In [292]:
dataframes = {}
#loop through the list of files and read them into pandas, concatenating each list to one dataframe
for file in incident_list:
    df = pd.read_csv('../theft_and_crime_clt/Data_input/Incident/'+file)
    dataframes[file] = df
incident_df = pd.concat(dataframes, ignore_index=True)
dataframes = {}
for file in offense_list:
    df = pd.read_csv('../theft_and_crime_clt/Data_input/Offense/'+file)
    dataframes[file] = df
offense_df = pd.concat(dataframes, ignore_index=True)
dataframes = {}
for file in property_list:
    df = pd.read_csv('../theft_and_crime_clt/Data_input/Property/'+file)
    dataframes[file] = df
property_df = pd.concat(dataframes, ignore_index=True)
dataframes = {}
for file in vehicle_list:
    df = pd.read_csv('../theft_and_crime_clt/Data_input/Vehicle/'+file)
    dataframes[file] = df
vehicle_df = pd.concat(dataframes, ignore_index=True)
dataframes = {}
for file in business_list:
    df = pd.read_csv('../theft_and_crime_clt/Data_input/Business/'+file)
    dataframes[file] = df
business_df = pd.concat(dataframes, ignore_index=True)
dataframes = {}

C:\Users\jesse\AppData\Local\Temp\ipykernel_46384\1982941201.py:4: DtypeWarning: Columns (17,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../theft_and_crime_clt/Data_input/Incident/'+file)


Making sure everything is numeric

In [293]:
#find what is not of type int or float in the column
business_df[~business_df['Complaint_No'].apply(lambda x: isinstance(x, (int, float)))]
business_df['Complaint_No'] = pd.to_numeric(business_df['Complaint_No'], errors='coerce')
business_df[~business_df['Complaint_No'].apply(lambda x: isinstance(x, (int, float)))]
business_df['Complaint_No'].astype(float, errors='raise')
business_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222667 entries, 0 to 222666
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Complaint_No            212522 non-null  float64
 1   Victim_Bus_No           222667 non-null  int64  
 2   Victim_Bus_LegalName    222667 non-null  object 
 3   Victim_Bus_Name         222667 non-null  object 
 4   Victim_Bus_Type         222667 non-null  object 
 5   Victim_Bus_Block_No     222667 non-null  object 
 6   Victim_Bus_Direction    222667 non-null  object 
 7   Victim_Bus_Street_Name  222667 non-null  object 
 8   Victim_Bus_Suffix       222667 non-null  object 
 9   Victim_Bus_Apt_No1      222667 non-null  object 
 10  Victim_Bus_Apt_No2      222667 non-null  object 
 11  Victim_Bus_City         222667 non-null  object 
 12  Victim_Bus_State        222667 non-null  object 
 13  Victim_Bus_ZipCode      222667 non-null  object 
 14  Unnamed: 14         

In [294]:
pd.to_numeric(incident_df['Complaint_No'], errors='coerce')
print(incident_df['Complaint_No'].describe())

pd.to_numeric(offense_df['Complaint_No'], errors='coerce')
print(offense_df['Complaint_No'].describe())

pd.to_numeric(property_df['Complaint_No'], errors='coerce')
print(property_df['Complaint_No'].describe())

pd.to_numeric(vehicle_df['Complaint_No'], errors='coerce')
print(vehicle_df['Complaint_No'].describe())

count    5.667890e+05
mean     2.013620e+13
std      1.674938e+10
min      2.011010e+13
25%      2.012083e+13
50%      2.013123e+13
75%      2.015073e+13
max      2.016123e+13
Name: Complaint_No, dtype: float64
count    1.028777e+06
mean     2.013340e+13
std      1.577549e+10
min      2.011010e+13
25%      2.012073e+13
50%      2.013062e+13
75%      2.015021e+13
max      2.016123e+13
Name: Complaint_No, dtype: float64
count    1.145838e+06
mean     2.013325e+13
std      1.582121e+10
min      2.011010e+13
25%      2.012072e+13
50%      2.013062e+13
75%      2.015013e+13
max      2.016123e+13
Name: Complaint_No, dtype: float64
count    2.991900e+04
mean     2.013366e+13
std      1.624224e+10
min      2.011010e+13
25%      2.012072e+13
50%      2.013063e+13
75%      2.015041e+13
max      2.016123e+13
Name: Complaint_No, dtype: float64


Dropping NaNs in Complaint_No for all

In [295]:
#print count na Complaint_No
print(incident_df['Complaint_No'].isna().sum())
print(offense_df['Complaint_No'].isna().sum())
print(property_df['Complaint_No'].isna().sum())
print(vehicle_df['Complaint_No'].isna().sum())
print(business_df['Complaint_No'].isna().sum())

0
0
0
0
10145


In [296]:
incident_df.dropna(subset=['Complaint_No'], inplace=True)
offense_df.dropna(subset=['Complaint_No'], inplace=True)
property_df.dropna(subset=['Complaint_No'], inplace=True)
vehicle_df.dropna(subset=['Complaint_No'], inplace=True)
business_df.dropna(subset=['Complaint_No'], inplace=True)

In [297]:
incident_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566789 entries, 0 to 566788
Data columns (total 30 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Complaint_No         566789 non-null  int64  
 1   Block_No             566789 non-null  object 
 2   Direction            566789 non-null  object 
 3   Street_Name          566789 non-null  object 
 4   Street_Type          566788 non-null  object 
 5   Suffix               566789 non-null  object 
 6   Apt_No               566789 non-null  object 
 7   City                 566789 non-null  object 
 8   State                566789 non-null  object 
 9   Zipcode              566789 non-null  object 
 10  X_Coordinate         566789 non-null  object 
 11  Y_Coordinate         566789 non-null  object 
 12  Division             566789 non-null  object 
 13  Reported_Date        566789 non-null  object 
 14  Location_Desc        566788 non-null  float64
 15  Location_Type    

In [298]:
incident_df.drop_duplicates

<bound method DataFrame.drop_duplicates of           Complaint_No Block_No Direction  \
0       20110101000308     4425             
1       20110101000700     2228             
2       20110101001104     2300        N    
3       20110101001302     4027             
4       20110101002401      150        S    
...                ...      ...       ...   
566784  20161231234101      118        S    
566785  20161231234200     7321        N    
566786  20161231234901     3826             
566787  20161231235000     2811             
566788  20161231235500     1314             

                                     Street_Name Street_Type    Suffix  \
0       EDDLEMAN                                          RD             
1       BEATTIES FORD                                     RD             
2       TRYON                                             ST             
3       QUAIL GLENN                                       CT             
4       COLLEGE                               

In [301]:
offense_df.drop_duplicates(subset=['Complaint_No'])
offense_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1028777 entries, 0 to 1028776
Data columns (total 5 columns):
 #   Column                    Non-Null Count    Dtype 
---  ------                    --------------    ----- 
 0   Complaint_No              1028777 non-null  int64 
 1   off_Attempt_Complete      1028777 non-null  object
 2   Off_nibrs_crime_category  1028777 non-null  object
 3   off_nibrs_classification  1028777 non-null  object
 4   off_noc_code              1028777 non-null  object
dtypes: int64(1), object(4)
memory usage: 39.2+ MB


In [310]:
offense_df['Complaint_No'].

<bound method Series.drop_duplicates of 0          20110101000308
1          20110101000308
2          20110101000700
3          20110101000700
4          20110101000700
                ...      
1028772    20161231234200
1028773    20161231234901
1028774    20161231235000
1028775    20161231235500
1028776    20161231235500
Name: Complaint_No, Length: 1028777, dtype: int64>

In [311]:
#find a complaint number with multiple rows
offense_df['Complaint_No'].value_counts()

20121019114201    114
20121023134604     75
20121218091400     72
20131002053900     70
20130306070903     60
                 ... 
20150115134203      1
20110816154103      1
20110816154100      1
20150115135201      1
20140304140802      1
Name: Complaint_No, Length: 539641, dtype: int64

In [270]:
offense_df[offense_df['Complaint_No'] == 20121019114201]

,Complaint_No,off_Attempt_Complete,Off_nibrs_crime_category,off_nibrs_classification,off_noc_code
204680,20121019114201,Attempted,Larceny / B/E Vehicle (larceny from) ...,Theft From Motor Vehicle ...,23F
204681,20121019114201,Attempted,Larceny / B/E Vehicle (larceny from) ...,Theft From Motor Vehicle ...,23F
204682,20121019114201,Attempted,Larceny / B/E Vehicle (larceny from) ...,Theft From Motor Vehicle ...,23F
204683,20121019114201,Attempted,Larceny / B/E Vehicle (larceny from) ...,Theft From Motor Vehicle ...,23F
204684,20121019114201,Attempted,Larceny / B/E Vehicle (larceny from) ...,Theft From Motor Vehicle ...,23F
...,...,...,...,...,...
501011,20121019114201,Completed,Larceny / B/E Vehicle (larceny from) ...,Theft From Motor Vehicle ...,23F
501012,20121019114201,Completed,Larceny / B/E Vehicle (larceny from) ...,Theft From Motor Vehicle ...,23F
501013,20121019114201,Completed,Larceny / B/E Vehicle (larceny from) ...,Theft From Motor Vehicle ...,23F
501014,20121019114201,Completed,Larceny / B/E Vehicle (larceny from) ...,Theft From Motor Vehicle ...,23F


In [306]:
print(incident_df['Complaint_No'].count())
print(offense_df['Complaint_No'].count())
data = pd.merge(incident_df, offense_df, on='Complaint_No', how='inner')
data.info()

566789
1028777
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1195139 entries, 0 to 1195138
Data columns (total 34 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   Complaint_No              1195139 non-null  int64  
 1   Block_No                  1195139 non-null  object 
 2   Direction                 1195139 non-null  object 
 3   Street_Name               1195139 non-null  object 
 4   Street_Type               1195138 non-null  object 
 5   Suffix                    1195139 non-null  object 
 6   Apt_No                    1195139 non-null  object 
 7   City                      1195139 non-null  object 
 8   State                     1195139 non-null  object 
 9   Zipcode                   1195139 non-null  object 
 10  X_Coordinate              1195139 non-null  object 
 11  Y_Coordinate              1195139 non-null  object 
 12  Division                  1195139 non-null  object 
 13  Reported_Dat

look at distribution of property and redo aggregate -- 
need a ton of census data about the environment -- household size, number of businesses, schools, groceries stores, etc. as much information about the location as possible...
